In [1]:
import argparse
import data
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
import Training
import QCNN_circuit

C:\ProgramData\anaconda3\Lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.0.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-pa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
import tensorflow

In [ ]:


def get_args():
	parser = argparse.ArgumentParser()	 

	parser.add_argument('--run_name', type=str, default="run_1", help='name of current run')

	parser.add_argument('--epochs', type=int, default=2, help='number of epochs')
	parser.add_argument('--batch_size', type=int, default=64, help='number of elements in batch size')

	parser.add_argument('--lr', type=float, default=0.01, help='learning rate')
	parser.add_argument('--opt', type=str, default='Adam', choices=['QNGO', 'Adam', 'GDO'], help = 'optimizer used for training')

	parser.add_argument('--loss', type=str, default = 'cross_entropy', choices=['cross_entropy'], help='loss function')
	
	parser.add_argument('--num_classes', type=int, default = 10, choices=[4, 6, 8, 10], help='num classes')
	
	parser.add_argument('--encoding', type=str, default = "amplitude", choices=['amplitude', 'angle'], help='encoding type')

	parser.add_argument('--num_layer', type=int, default = 1, choices=range(1, 10), help = 'number of circuit repetition')
	parser.add_argument('--load_params', type=bool, default=False, choices=[True, False], help='continue training from saved parameters')

	parser.add_argument('--all_samples', type=bool, default=False, choices=[False, True], help="how many samples of the dataset")

	parser.add_argument('--seed', type = int, default = None, help = "set seed for reproducibility, if not set, it is random")


	return parser.parse_args()
	

def accuracy_multi(predictions, labels, num_classes):
	correct_predictions = 0

	
	for l, p in zip(labels, predictions):
		p2 = []
		for i in range(0, num_classes):
			p2.append(p[i])
		predicted_class = np.argmax(p2)	# Find the index of the predicted class with highest probability
		if predicted_class == l:
			correct_predictions += 1

	accuracy = correct_predictions / len(labels)
	return accuracy

def accuracy_test_multiclass(predictions, label, num_classes):
	#confusion matrix
	
	preds_np = np.array(predictions)
	preds = np.argmax(preds_np[:, :num_classes], axis = 1)
	
	conf_mat = multilabel_confusion_matrix(label, preds, labels = list(range(num_classes)))
	print(conf_mat)
	precision = []
	recall = []
	f1 = []
	i = 0
	for c in conf_mat:
		precision.append(c[1][1] / (c[1][1] + c[0][1]))
		recall.append(c[1][1] / (c[1][1] + c[1][0]))
		f1.append(2 * (precision[i] * recall[i]) / (precision[i] + recall[i] + np.finfo(float).eps))
		
		print("precision " + str(i) + ": " + str(precision[i])) 
		print("recall " + str(i) + ": " + str(recall[i])) 
		print("f1 " + str(i) + ": " + str(f1[i])) 
		i += 1

def main(args):
	
	U_params = 15

	X_train, X_test, Y_train, Y_test = data.data_load_and_process(args.encoding, args.num_classes, args.all_samples, args.seed)

	print("\n")
	print("Loss History for  circuit with " + args.encoding + " with " + args.loss)
	loss_history, trained_params = Training.circuit_training(X_train, Y_train, U_params, args.encoding, args.loss, args.num_classes, args.num_layer, args.load_params,
															 args.opt, args.lr, args.epochs, args.all_samples, args.batch_size, args.seed)

	predictions = []
				
	for x in X_test:	
		predictions.append(QCNN_circuit.QCNN(x, trained_params, U_params, args.encoding, args.loss, args.num_classes, args.num_layer))
							
	accuracy = accuracy_multi(predictions, Y_test, args.num_classes)
	print("Accuracy: " + str(accuracy))
	accuracy_test_multiclass(predictions, Y_test, args.num_classes)

if __name__ == "__main__":
	args = get_args()
	print(args)
	main(args)

